# Notebook to solve the kaggle competition "Child Mind Institute - Detect Sleep States" with a neural network
Link to the competition: https://www.kaggle.com/competitions/child-mind-institute-detect-sleep-states


### Install packages

In [ ]:
!pip install matplotlib plotly pandas numpy tqdm scikit-learn pyarrow

To be able to switch between different models from different libraries at a glance, we implement an interface called `BaseFrameworkAbstraktion`. This allows us to make the pipleine even more robust and easier to extend upon. 

In [3]:
from abc import ABC, abstractmethod, classmethod

class BaseFrameworkAbstraction(ABC):
    @abstractmethod
    def a(self):
        pass

Our first model is the baseline model, which just takes the mean over all onset and wakeup times and tries to predict `onset` and `wakeup` events with the calculated time.

In [9]:
class BaselineModel(BaseFrameworkAbstraction):
    def __init__(self):
        pass

    def a(self):
        raise NotImplementedError()

In [10]:
g = BaselineModel()